In [14]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm

In [18]:
df=load_iris()
x=df.data
y=df.target
x=sm.add_constant(x,prepend=True)

In [25]:
model=sm.OLS(y,x)
res=model.fit()

In [26]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     484.5
Date:                Wed, 28 Aug 2024   Prob (F-statistic):           8.46e-83
Time:                        11:07:24   Log-Likelihood:                 17.437
No. Observations:                 150   AIC:                            -24.87
Df Residuals:                     145   BIC:                            -9.821
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1865      0.205      0.910      0.364      -0.218       0.591
x1            -0.1119      0.058     -1.941      0.054      -0.226       0.002
x2            -0.0401      0.060     -0.671      0.503      -0.158       0.078
x3             0.2286      0.057      4.022      0.000       0.116       0.341
x4             0.6093      0.094      6.450      0.000       0.423       0.796
==============================================================================
Omnibus:                        0.374   Durbin-Watson:                   1.077
Prob(Omnibus):                  0.829   Jarque-Bera (JB):                0.141
Skew:                          -0.051   Prob(JB):                        0.932
Kurtosis:                       3.110   Cond. No.                         91.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
res